In [85]:
from sqlalchemy import create_engine
import pymysql
from datetime import datetime,timedelta
import tushare as ts
import pandas as pd 
pro=ts.pro_api()

#给出回溯天数，算出开始时间，以列表形式返回开始到现在的所有交易日期，日期格式为20191120.
def lmh_trade_dates(ndays):
    day=datetime.today()
    #取今天的时间
    end_date=day.strftime('%Y%m%d')
    #默认输出是2019-10-24的格式，用strftime换算成20191024
    start_date=(day-timedelta(days=ndays)).strftime('%Y%m%d')
    #开始时间根据ndays的不同，可以变化
    cal_date=pro.trade_cal(exchange='', start_date=start_date, end_date=end_date)
    #获取各大交易所交易日历数据,默认提取的是上交所
    trade_days=cal_date[cal_date.is_open==1].cal_date
    #is_open=1，表明该日期开市交易.trade_date为为指定时间内的交易日列表
    return list(trade_days)
    #返回类型是pandas.core.series.Series，可以用list转化为列表
    
#读出指定日期的mysql数据    
def stock_min_data(trade_dates,stockcode):
    datalist=[]
    for dates in trade_dates:
        minnow=datetime.today().strftime('%H%M')
        #当前时间的小时分钟数
        tablename='min'+dates+minnow
        dbname='stockmin'+dates[0:6]
        #print(dbname)
        db = pymysql.connect("rememberme.top","root","prophet2lmh",dbname)
        #建立数据库连接
        cursor = db.cursor()
        #建立游标
        
        
        sqlcommand='show columns from '+tablename+';'
        cursor.execute(sqlcommand)
        data = cursor.fetchall()
        column_name=[]
        for i in data:
            column_name.append(i[0])
        #查询表格的列名并保存到列表中
        
        
    
        sqlcommand='select * from '+tablename+' where 代码='+stockcode+';'
        cursor.execute(sqlcommand)       
        data=cursor.fetchone()
        # 使用 fetchone() 方法获取单条数据.
        datalist.append(list(data)[1:])
        #将每次的数据增加到datalist中
        
        db.close()
        # 关闭数据库连接
        
    df=pd.DataFrame(datalist,columns=column_name[1:])
    return df


stock_min_data(lmh_trade_dates(2),'600397')


,代码,名称,最新,涨幅%,涨跌,总手,现手,买入价,卖出价,涨速%,...,每股收益,每股净资产,营业利润,总资产,总负债,营业总收入,营业总收入同比,每股公积金,利润总额,采集时间
0,600397,安源煤业,2.00,2.56,0.05,6.79万,60,2.00,2.01,0.00,...,-0.09,0.77,-6456万,64.4亿,57.9亿,39.9亿,13.71,1.88,-8654万,20191118145243
1,600397,安源煤业,2.06,3.00,0.06,4.28万,27,2.05,2.06,0.00,...,-0.09,0.77,-6456万,64.4亿,57.9亿,39.9亿,13.71,1.88,-8654万,20191119133916
2,600397,安源煤业,2.27,10.19,0.21,19.3万,100,2.27,----,0.00,...,-0.09,0.77,-6456万,64.4亿,57.9亿,39.9亿,13.71,1.88,-8654万,20191120145219


In [82]:
b

,代码,名称,最新,涨幅%,涨跌,总手,现手,买入价,卖出价,涨速%,...,每股收益,每股净资产,营业利润,总资产,总负债,营业总收入,营业总收入同比,每股公积金,利润总额,采集时间
0,600397,安源煤业,1.97,1.03,0.02,1.69万,100,1.97,1.98,-0.51,...,-0.09,0.77,-6456万,64.4亿,57.9亿,39.9亿,13.71,1.88,-8654万,20191118100446
1,600397,安源煤业,2.02,1.00,0.02,1.14万,1,2.01,2.02,0.00,...,-0.09,0.77,-6456万,64.4亿,57.9亿,39.9亿,13.71,1.88,-8654万,20191119100516
2,600397,安源煤业,2.24,8.74,0.18,12.8万,11,2.23,2.24,-0.88,...,-0.09,0.77,-6456万,64.4亿,57.9亿,39.9亿,13.71,1.88,-8654万,20191120100520
